## 中文

In [1]:
import logging
import collections
import re
import math
import jieba
from wordcloud import WordCloud
jieba.setLogLevel(logging.INFO)
import os

root_path = '/Users/zhongshannan/Documents/fakenews_detection/dataset/weibo数据集处理/'

def stopwordslist():
    stopwords = [line.strip() for line in open(os.path.join(root_path, 'stop_words.txt'), encoding='UTF-8').readlines()]
    stopwords = [k for s in stopwords for k in s]
    return stopwords

f = open(os.path.join(root_path, '事件29.txt'), "r", encoding="utf-8")
text = f.read()
f.close()

TopWordFrequencyPercentage = 20
TopWordFrequencyPercentage /= 100

# jieba分词
seg = jieba.cut(text)
seg = " ".join(seg)
seg = seg.strip()
seg = re.sub(r"[0-9\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——！，;:：。？、~@#￥%……&*（）]+", " ", seg)
seg = seg.split(" ")
seg = list(filter(None, seg))
stopwords = stopwordslist()
seg = [v for v in seg if v not in stopwords]
word_counts = collections.Counter(seg)

from matplotlib.colors import LinearSegmentedColormap
colors = ["#000000", "#111111", "#101010", "#121212", "#212121", "#222222"]
cmap = LinearSegmentedColormap.from_list("mycmap", colors)

# 生成词云
wc = WordCloud(
    max_words=math.ceil(len(word_counts) * TopWordFrequencyPercentage),
    width=500,
    height=350,
    max_font_size=50,
    min_font_size=10,
    font_path='/System/Library/Fonts/Supplemental/Songti.ttc',
    background_color='white',
    colormap=cmap,
)

wc.generate_from_frequencies(word_counts)

PicSavePath = "事件29.png"
wc.to_file(PicSavePath)

## 英文

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
import string
import seaborn as sns

trainTweets = pd.read_csv("/Users/zhongshannan/Documents/fakenews_detection/dataset/twitter/twitter_cleaned/train_posts.csv")

# 转小写
trainTweets['post_text'] = trainTweets['post_text'].apply(lambda x: x.lower())
trainTweets.head()

# 删除标点
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

trainTweets['post_text'] = trainTweets['post_text'].apply(punctuation_removal)
trainTweets.head()

# 删除英文停用词
nltk.download('stopwords')
stop = stopwords.words('english')
trainTweets['post_text'] = trainTweets['post_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


en_stop_words = nltk.corpus.stopwords.words('english')
sp_stop_words = nltk.corpus.stopwords.words('spanish')
twitter = ['rt', 'http', 'co','https','nhttp','nhttps']

# 完整停用词 由英文停用词、西班牙文停用词、事件词汇、rt构成
block_words = en_stop_words + sp_stop_words + twitter

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

# 自定义图层颜色
from matplotlib.colors import LinearSegmentedColormap
colors = ["#000000", "#111111", "#101010", "#121212", "#212121", "#222222"]
cmap = LinearSegmentedColormap.from_list("mycmap", colors)

def generate_word_cloud(input_tweets):
    tweets = input_tweets
    tweets_text = " ".join(tweets.post_text.to_numpy().tolist())
    
    # 词云
    wordcloud = WordCloud(width = 3000, 
                          height = 2000, 
                          random_state=1, 
                          background_color='White',
                          colormap=cmap, 
                          collocations=False, 
                          font_path='/System/Library/Fonts/Supplemental/Marion.ttc',
                          stopwords = block_words).generate(tweets_text)
    # plt.figure(figsize=(16,10))
    # plt.imshow(wordcloud, interpolation='bilinear')
    # plt.axis('off') # 关闭轴
    # plt.show()
    wordcloud.to_file("7.png")

generate_word_cloud(trainTweets[trainTweets['label'] == 'real'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhongshannan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
